DyslexiaLens

Text Ingestion

In [4]:
#text = input(Text Input: )
text = """In many cities today, large construction projects often make life harder for poor and vulnerable people. These projects can push people out of their homes and make it harder to get basic services like water, transport, or healthcare.
Although city planners say they want to build in a fair and sustainable way, they often focus more on money than on including everyone in the process. As a result, the needs of the most at-risk communities are often ignored."""


Preprocessing

1. Text Cleaning

In [5]:
import ftfy
import re

In [6]:
def clean_text(text):

    #to clear the encoding issues
    text = ftfy.fix_text(text)

    #to remove the non printable letters
    text = ''.join(c for c in text if c.isprintable())
    
    #to remove the extra spaces
    text = re.sub(r'[\r\n]+', '\n', text)
    text = re.sub(r'\s{2,}', ' ', text)

    return text.strip()

In [7]:
cleanText = clean_text(text)

2. Segmentation

In [8]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/prathoseraaj-v/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
def segmentation_text(text):
    paragraph = [p for p in text.split('\n') if p.strip()]
    sentance = []

    for para in paragraph:
        sentance.extend(nltk.sent_tokenize(para))

    tokens = [nltk.word_tokenize(sent) for sent in sentance]
    return {
        'paragraph': paragraph,
        'sentances': sentance,
        'tokens': tokens,
    }

In [10]:
segmentation_text(text)

{'paragraph': ['In many cities today, large construction projects often make life harder for poor and vulnerable people. These projects can push people out of their homes and make it harder to get basic services like water, transport, or healthcare.',
  'Although city planners say they want to build in a fair and sustainable way, they often focus more on money than on including everyone in the process. As a result, the needs of the most at-risk communities are often ignored.'],
 'sentances': ['In many cities today, large construction projects often make life harder for poor and vulnerable people.',
  'These projects can push people out of their homes and make it harder to get basic services like water, transport, or healthcare.',
  'Although city planners say they want to build in a fair and sustainable way, they often focus more on money than on including everyone in the process.',
  'As a result, the needs of the most at-risk communities are often ignored.'],
 'tokens': [['In',
   'm

Readability Assesment

In [11]:
import spacy

In [12]:
nlp = spacy.load("en_core_web_sm")

1. Readability Score

In [13]:
import textstat

In [14]:
def readability_score(text):
    return{
        "flesch_reading_ease" : textstat.flesch_reading_ease(text),
        "flesch_kincaid_grade" : textstat.flesch_kincaid_grade(text),
        "gunning_fog": textstat.gunning_fog(text),
        "smog_index": textstat.smog_index(text),
        "coleman_liau_index" : textstat.coleman_liau_index(text),
        "automated_readability_index" : textstat.automated_readability_index(text),
        "dale_chall_readability_score" : textstat.dale_chall_readability_score(text), 
            }

In [ ]:
readability_score(text)

{'flesch_reading_ease': 57.97326923076926,
 'flesch_kincaid_grade': 10.017564102564105,
 'gunning_fog': 10.876923076923077,
 'smog_index': 10.686352973137792,
 'coleman_liau_index': 10.26923076923077,
 'automated_readability_index': 11.326538461538462,
 'dale_chall_readability_score': 9.259725641025641}

2. Detect long sentance

In [ ]:
def detect_long_sentance(sentances, threshold=25):
    return [sent for sent in sentances if len(sent.split()) > threshold]

3. Detect passive voice

In [3]:
import spacy

In [4]:
def detect_passive_voice(sentences):
    passive_sentences = []

    for sent in sentences:
        doc = nlp(sent)
        for token in doc:
            if token.dep_ == "nsubjpass":
                passive_sentences.append(sent)
                break

    return passive_sentences

4. Detect ambigious structures

In [ ]:
def detect_ambiguous_structures(sentences):
    ambiguous_keywords = ["might", "could", "possibly", "maybe", "potentially", "approximately", "suggests", "appears"]
    return [sent for sent in sentences if any(word in sent.lower() for word in ambiguous_keywords)]